# MSHNet - Google Colab A100 训练脚本
## 红外小目标检测 - 尺度和位置敏感性

### 📌 注意事项：
1. **选择A100 GPU**：运行时 → 更改运行时类型 → 硬件加速器 → GPU (A100)
2. **数据集准备**：确保已上传数据集或使用Google Drive挂载
3. **A100优势**：40GB显存，可以使用更大的batch size
4. **会话时长**：Colab Pro有更长的会话时间，建议使用

### ⚠️ 重要提示：
- 定期保存权重到Google Drive
- 使用checkpoint功能避免训练中断
- 监控GPU使用情况


## 1. 检查GPU信息


In [ ]:
# 检查GPU信息
!nvidia-smi
print('\n' + '='*50)

import torch
print(f'PyTorch版本: {torch.__version__}')
print(f'CUDA可用: {torch.cuda.is_available()}')
print(f'CUDA版本: {torch.version.cuda}')
print(f'GPU数量: {torch.cuda.device_count()}')
if torch.cuda.is_available():
    print(f'GPU名称: {torch.cuda.get_device_name(0)}')
    print(f'GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB')


## 2. 克隆项目代码


In [ ]:
# 方法1: 克隆GitHub仓库（替换为您的仓库地址）
!git clone https://github.com/your-username/MSHNet.git
%cd MSHNet

# 方法2: 从Google Drive复制项目
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r /content/drive/MyDrive/MSHNet /content/
# %cd /content/MSHNet


## 3. 安装依赖


In [ ]:
# 安装必要的依赖包（Colab已预装PyTorch）
!pip install -q scikit-image tqdm

# 验证所有依赖
import numpy as np
import torch
from PIL import Image
from skimage import measure
from tqdm import tqdm
print('✓ 所有依赖安装成功！')


## 4. 挂载Google Drive并准备数据集


In [ ]:
# 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 如果数据集已在Drive中，创建软链接
!mkdir -p datasets
!ln -s /content/drive/MyDrive/datasets/IRSTD-1k ./datasets/IRSTD-1k

# 或者解压数据集
# !tar -xzf /content/drive/MyDrive/IRSTD-1k.tar.gz -C ./datasets/

# 检查数据集结构
!ls -lh datasets/IRSTD-1k/


## 5. 开始训练（A100优化参数）


In [ ]:
# A100优化训练参数
!python main.py \
    --dataset-dir './datasets/IRSTD-1k' \
    --batch-size 16 \
    --epochs 400 \
    --lr 0.05 \
    --base-size 256 \
    --crop-size 256 \
    --warm-epoch 5 \
    --mode train

# 注意：batch-size从4增加到16，充分利用A100的40GB显存


## 6. 定期保存权重到Google Drive


In [ ]:
# 创建保存目录并复制权重
!mkdir -p /content/drive/MyDrive/MSHNet_results

# 复制最新的权重文件
!cp -r /MSHNet/weight/* /content/drive/MyDrive/MSHNet_results/

# 打包所有权重文件
!tar -czf /content/drive/MyDrive/MSHNet_results/weights_backup.tar.gz /MSHNet/weight/

print('✓ 权重已保存到Google Drive')


## 7. 测试模型


In [ ]:
# 使用训练好的权重进行测试
!python main.py \
    --dataset-dir './datasets/IRSTD-1k' \
    --batch-size 1 \
    --mode test \
    --weight-path './IRSTD-1k_weight.tar'


## 8. 监控训练进度（可选）


In [ ]:
# 实时查看训练日志
import glob
import os

# 找到最新的训练目录
weight_dirs = glob.glob('./weight/MSHNet-*')
if weight_dirs:
    latest_dir = max(weight_dirs, key=os.path.getctime)
    print(f'最新训练目录: {latest_dir}')
    
    # 查看训练日志
    log_file = os.path.join(latest_dir, 'metric.log')
    if os.path.exists(log_file):
        print('\n最近10条训练记录：')
        !tail -10 {log_file}
else:
    print('未找到训练目录')
